# Building the model

The purpose of this notebook is to build the model that will eventually be used as the model behind the PL prediction for the app, and building the script to rebuild the model when new player data is introduced weekly

## Imports

In [1]:
import pandas as pd
import numpy as np
from sklearn.decomposition import PCA
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import balanced_accuracy_score
from scikeras.wrappers import KerasRegressor
import tensorflow as tf
from sklearn.model_selection import GridSearchCV
from joblib import dump, load

## Loading the dataset

In [2]:
combined = pd.read_csv("../files/final_combined_dataframe.csv")

In [3]:
combined.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2116 entries, 0 to 2115
Data columns (total 94 columns):
 #   Column                                  Non-Null Count  Dtype  
---  ------                                  --------------  -----  
 0   match_id                                2116 non-null   object 
 1   goals                                   2116 non-null   float64
 2   assists                                 2116 non-null   float64
 3   direct_goal_contributions               2116 non-null   float64
 4   non_penalty_goals                       2116 non-null   float64
 5   penalties_scored                        2116 non-null   float64
 6   penalties_attempted                     2116 non-null   float64
 7   yellow_cards                            2116 non-null   float64
 8   red_cards                               2116 non-null   float64
 9   expected_goals                          2116 non-null   float64
 10  non_penalty_expected_goals              2116 non-null   floa

In [4]:
combined.head()

,match_id,goals,assists,direct_goal_contributions,non_penalty_goals,penalties_scored,penalties_attempted,yellow_cards,red_cards,expected_goals,...,away_fouls,home_yellow_cards,away_yellow_cards,home_red_cards,away_red_cards,home_team_at_home_mean_goal_difference,home_team_overall_mean_goal_difference,away_team_at_away_mean_goal_difference,away_team_overall_mean_goal_difference,head_to_head_goal_difference
0,m-00381,0.393401,0.269112,0.662513,0.355397,0.038004,0.054601,0.556054,-0.031945,0.394375,...,9,2,2,0,0,2.4,1.6,-2.8,0.2,7
1,m-00382,0.134401,0.408515,0.542916,0.169614,-0.035212,-0.062705,0.464746,0.040671,0.203698,...,14,5,2,0,0,1.4,1.6,-0.6,1.2,1
2,m-00383,1.032333,0.289712,1.322045,0.976505,0.055828,0.080853,0.654894,-0.011413,0.903525,...,14,2,5,0,0,1.0,1.6,-1.4,0.2,8
3,m-00384,0.073795,0.225465,0.299260,0.152876,-0.079081,-0.080597,0.105934,-0.043515,-0.051271,...,15,0,2,0,0,2.0,1.6,0.0,0.6,1
4,m-00385,-0.308318,-0.074280,-0.382598,-0.265916,-0.042402,-0.029883,0.243744,-0.032462,-0.213221,...,12,4,1,0,0,1.8,1.6,1.0,0.0,3


## Column Lists

In [5]:
output_columns = [
	"home_goals", "away_goals", "home_shots", "away_shots", "home_shots_on_target", "away_shots_on_target",
	"home_corners", "away_corners", "home_fouls", "away_fouls", "home_yellow_cards", "away_yellow_cards",
	"home_red_cards", "away_red_cards"
]
match_columns = [
	"match_id", "competition_id", "home_team_id", "away_team_id", "referee_id",
	"home_goals", "away_goals", "home_shots", "away_shots", "home_shots_on_target", "away_shots_on_target",
	"home_corners", "away_corners", "home_fouls", "away_fouls", "home_yellow_cards", "away_yellow_cards",
	"home_red_cards", "away_red_cards"
]
stats_columns = [
	"goals","assists","penalties_scored","penalties_attempted","yellow_cards","red_cards","expected_goals",
	"non_penalty_expected_goals","expected_assisted_goals","progressive_carries","progressive_passes","total_passing_distance",
	"total_progressive_passing_distance","short_passes_completed","short_passes_attempted","medium_passes_completed",
	"medium_passes_attempted","long_passes_completed","long_passes_attempted","expected_assists","key_passes",
	"passes_into_final_third","passes_into_penalty_area","crosses_into_penalty_area","shots","shots_on_target",
	"average_shot_distance","shots_from_free_kicks","touches_in_defensive_penalty_area","touches_in_defensive_third",
	"touches_in_middle_third","touches_in_attacking_third","touches_in_attacking_penalty_area","live_ball_touches",
	"take_ons_attempted","take_ons_succeeded","carries","total_carrying_distance","progressive_carrying_distance",
	"carries_into_final_third","carries_into_penalty_area","miscontrols","dispossessed","passes_received",
	"progressive_passes_received","tackles_won","defensive_third_tackles","middle_third_tackles","attacking_third_tackles",
	"dribblers_tackled","dribbler_tackles_attempted","shots_blocked","passes_blocked","interceptions","clearances",
	"errors_leading_to_shot","goals_against","shots_on_target_against","saves","clean_sheets","penalties_faced",
	"penalties_allowed","penalties_saved","penalties_missed","home_team_at_home_mean_goal_difference",
	"home_team_overall_mean_goal_difference","away_team_at_away_mean_goal_difference","away_team_overall_mean_goal_difference",
	"head_to_head_goal_difference"
]

player_stats_columns = ["player_id", "minutes_played","ninetys"] + stats_columns
pure_stats_columns = ["minutes_played"] + stats_columns
team_stats_columns = ["team_id"] + stats_columns


## Scaling

In [6]:
scaler = StandardScaler(copy=True)

In [7]:
combined_standardized = scaler.fit_transform(combined[stats_columns])

In [8]:
evr = {}
n_components = {}
feature_to_pc = {}

for n in range(1,26):
	pca = PCA(n_components = n, random_state=938)
	pca.fit(combined_standardized)
	feature_to_pc_map = pd.DataFrame(pca.components_, columns=stats_columns)
	components = pca.transform(combined_standardized)
	components_df = pd.DataFrame(data=components[:, [p for p in range(n)]], columns=pca.get_feature_names_out(), )
	
	evr[n] = sum(pca.explained_variance_ratio_)
	n_components[n] = components_df
	feature_to_pc[n] = feature_to_pc_map

pd.DataFrame(data=evr, index=["explained_variance_ratio"]).T
feature_to_pc[15].to_csv("../files/feature_to_15_pcs.csv")


Check the effect of different numbers if PCs on the outcome of the NN, testing n=2, 5 and 10. Use each n components to train and test the neural network, and compare the performance of each one

## Number of Features

In [9]:
INPUT_SIZE = len(stats_columns)
OUTPUT_SIZE = 14
ROWS = 1927

### n = 2

Train test split of data

In [10]:
n=2
X = n_components[n][['pca'+ str(x) for x in range(n)]]
Y = combined[output_columns]

X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size=0.2, random_state=629) 

Model definition

In [11]:
model = tf.keras.models.Sequential([
	tf.keras.layers.Dense(len(X_train.columns), activation='relu', input_dim=len(X_train.columns)),
	tf.keras.layers.Dense(128, activation='relu'),
	tf.keras.layers.Dense(OUTPUT_SIZE)
])
model.compile(loss="mean_squared_error", optimizer="sgd", metrics="accuracy")
model.fit(X_train, y_train, epochs=500, batch_size=50);

Epoch 1/500
34/34 [==============================] - 0s 427us/step - loss: 30.4561 - accuracy: 0.4326
Epoch 2/500
34/34 [==============================] - 0s 395us/step - loss: 21.9304 - accuracy: 0.4740
Epoch 3/500
34/34 [==============================] - 0s 316us/step - loss: 14.8294 - accuracy: 0.4740
Epoch 4/500
 1/34 [..............................] - ETA: 0s - loss: 10.5754 - accuracy: 0.4600

2024-04-29 09:49:02.025617: W tensorflow/tsl/platform/profile_utils/cpu_utils.cc:128] Failed to get CPU frequency: 0 Hz


34/34 [==============================] - 0s 344us/step - loss: 9.3353 - accuracy: 0.4740
Epoch 5/500
34/34 [==============================] - 0s 335us/step - loss: 9.0202 - accuracy: 0.4740
Epoch 6/500
34/34 [==============================] - 0s 333us/step - loss: 9.1301 - accuracy: 0.4740
Epoch 7/500
34/34 [==============================] - 0s 322us/step - loss: 8.9501 - accuracy: 0.4740
Epoch 8/500
34/34 [==============================] - 0s 335us/step - loss: 9.0932 - accuracy: 0.4740
Epoch 9/500
34/34 [==============================] - 0s 318us/step - loss: 8.7248 - accuracy: 0.4740
Epoch 10/500
34/34 [==============================] - 0s 320us/step - loss: 8.6164 - accuracy: 0.4740
Epoch 11/500
34/34 [==============================] - 0s 320us/step - loss: 8.7072 - accuracy: 0.4740
Epoch 12/500
34/34 [==============================] - 0s 319us/step - loss: 8.6590 - accuracy: 0.4740
Epoch 13/500
34/34 [==============================] - 0s 319us/step - loss: 8.5536 - accuracy: 0.474

In [12]:
y_hat = model.predict(X_test)
print(y_hat[1])
y_test.head()

14/14 [==============================] - 0s 390us/step
[1.2444496e+00 1.6921959e+00 1.1631388e+01 1.3624737e+01 4.1153812e+00
 5.0046253e+00 4.7787991e+00 5.3916907e+00 1.0603401e+01 1.0504726e+01
 1.7295426e+00 1.5714474e+00 1.0978478e-01 9.5869740e-03]


,home_goals,away_goals,home_shots,away_shots,home_shots_on_target,away_shots_on_target,home_corners,away_corners,home_fouls,away_fouls,home_yellow_cards,away_yellow_cards,home_red_cards,away_red_cards
1634,2,1,13,16,4,3,3,5,8,10,1,1,0,0
1479,2,4,16,15,6,9,3,3,13,19,6,3,0,0
25,2,1,6,23,2,6,4,8,14,11,4,2,0,0
1686,2,1,12,17,4,5,6,8,11,7,3,1,0,0
1454,1,3,9,15,3,7,3,8,8,15,0,2,0,0


### n = 5

In [13]:
n=5
X = n_components[n][['pca'+ str(x) for x in range(n)]]
Y = combined[output_columns]

X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size=0.2, random_state=629) 

In [14]:
model = tf.keras.models.Sequential([
	tf.keras.layers.Dense(len(X_train.columns), activation='relu', input_dim=len(X_train.columns)),
	tf.keras.layers.Dense(128, activation='relu'),
	tf.keras.layers.Dense(OUTPUT_SIZE)
])
model.compile(loss="mean_squared_error", optimizer="sgd", metrics="accuracy")
model.fit(X_train, y_train, epochs=500, batch_size=50);

Epoch 1/500
34/34 [==============================] - 0s 410us/step - loss: 31.2005 - accuracy: 0.4243
Epoch 2/500
34/34 [==============================] - 0s 384us/step - loss: 15.2615 - accuracy: 0.4722
Epoch 3/500
34/34 [==============================] - 0s 340us/step - loss: 11.7087 - accuracy: 0.4669
Epoch 4/500
34/34 [==============================] - 0s 291us/step - loss: 9.9125 - accuracy: 0.4823
Epoch 5/500
34/34 [==============================] - 0s 353us/step - loss: 8.8169 - accuracy: 0.4900
Epoch 6/500
34/34 [==============================] - 0s 359us/step - loss: 8.2451 - accuracy: 0.5148
Epoch 7/500
34/34 [==============================] - 0s 356us/step - loss: 8.0392 - accuracy: 0.5390
Epoch 8/500
34/34 [==============================] - 0s 356us/step - loss: 8.0715 - accuracy: 0.5467
Epoch 9/500
34/34 [==============================] - 0s 341us/step - loss: 7.7551 - accuracy: 0.5455
Epoch 10/500
34/34 [==============================] - 0s 334us/step - loss: 7.7777 - acc

In [15]:
y_hat = model.predict(X_test)
print(y_hat[1])
y_test.head()

14/14 [==============================] - 0s 559us/step
[ 0.5321352   3.151731    9.296206   16.255259    2.7112641   6.9749446
  3.739766    5.73322     9.978015    9.656517    1.6812019   1.1554933
  0.21792987 -0.0464353 ]


,home_goals,away_goals,home_shots,away_shots,home_shots_on_target,away_shots_on_target,home_corners,away_corners,home_fouls,away_fouls,home_yellow_cards,away_yellow_cards,home_red_cards,away_red_cards
1634,2,1,13,16,4,3,3,5,8,10,1,1,0,0
1479,2,4,16,15,6,9,3,3,13,19,6,3,0,0
25,2,1,6,23,2,6,4,8,14,11,4,2,0,0
1686,2,1,12,17,4,5,6,8,11,7,3,1,0,0
1454,1,3,9,15,3,7,3,8,8,15,0,2,0,0


### n = 10

In [16]:
n=10
X = n_components[n][['pca'+ str(x) for x in range(n)]]
Y = combined[output_columns]

X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size=0.2, random_state=629) 

In [17]:
model = tf.keras.models.Sequential([
	tf.keras.layers.Dense(len(X_train.columns), activation='relu', input_dim=len(X_train.columns)),
	tf.keras.layers.Dense(128, activation='relu'),
	tf.keras.layers.Dense(OUTPUT_SIZE)
])
model.compile(loss="mean_squared_error", optimizer="sgd", metrics="accuracy")
model.fit(X_train, y_train, epochs=500, batch_size=50);

Epoch 1/500


34/34 [==============================] - 0s 407us/step - loss: 29.0764 - accuracy: 0.3942
Epoch 2/500
34/34 [==============================] - 0s 378us/step - loss: 14.2918 - accuracy: 0.4728
Epoch 3/500
34/34 [==============================] - 0s 349us/step - loss: 11.5114 - accuracy: 0.4953
Epoch 4/500
34/34 [==============================] - 0s 366us/step - loss: 9.9509 - accuracy: 0.5189
Epoch 5/500
34/34 [==============================] - 0s 346us/step - loss: 8.9901 - accuracy: 0.5384
Epoch 6/500
34/34 [==============================] - 0s 349us/step - loss: 8.4315 - accuracy: 0.5479
Epoch 7/500
34/34 [==============================] - 0s 351us/step - loss: 8.0499 - accuracy: 0.5502
Epoch 8/500
34/34 [==============================] - 0s 360us/step - loss: 7.8601 - accuracy: 0.5479
Epoch 9/500
34/34 [==============================] - 0s 340us/step - loss: 8.0625 - accuracy: 0.5514
Epoch 10/500
34/34 [==============================] - 0s 337us/step - loss: 7.7283 - accuracy: 0.553

In [18]:
y_hat = model.predict(X_test)
print(y_hat[1])
y_test.head()

14/14 [==============================] - 0s 414us/step
[5.8084112e-01 3.2481203e+00 1.0189079e+01 1.5303718e+01 2.7332897e+00
 7.0210066e+00 5.0651364e+00 5.3717790e+00 9.9045305e+00 8.9694796e+00
 1.3325444e+00 9.9260545e-01 7.6355301e-02 1.4490190e-02]


,home_goals,away_goals,home_shots,away_shots,home_shots_on_target,away_shots_on_target,home_corners,away_corners,home_fouls,away_fouls,home_yellow_cards,away_yellow_cards,home_red_cards,away_red_cards
1634,2,1,13,16,4,3,3,5,8,10,1,1,0,0
1479,2,4,16,15,6,9,3,3,13,19,6,3,0,0
25,2,1,6,23,2,6,4,8,14,11,4,2,0,0
1686,2,1,12,17,4,5,6,8,11,7,3,1,0,0
1454,1,3,9,15,3,7,3,8,8,15,0,2,0,0


### n=15

In [19]:
n=15
X = n_components[n][['pca'+ str(x) for x in range(n)]]
Y = combined[output_columns]

X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size=0.2, random_state=629) 

In [20]:
model = tf.keras.models.Sequential([
	tf.keras.layers.Dense(len(X_train.columns), activation='relu', input_dim=len(X_train.columns)),
	tf.keras.layers.Dense(128, activation='relu'),
	tf.keras.layers.Dense(OUTPUT_SIZE)
])
model.compile(loss="mean_squared_error", optimizer="sgd", metrics="accuracy")
model.fit(X_train, y_train, epochs=500, batch_size=50);

Epoch 1/500
34/34 [==============================] - 0s 465us/step - loss: 26.9006 - accuracy: 0.4037
Epoch 2/500
34/34 [==============================] - 0s 385us/step - loss: 13.7818 - accuracy: 0.4864
Epoch 3/500
34/34 [==============================] - 0s 361us/step - loss: 10.9110 - accuracy: 0.5000
Epoch 4/500
34/34 [==============================] - 0s 336us/step - loss: 9.7132 - accuracy: 0.5177
Epoch 5/500
34/34 [==============================] - 0s 358us/step - loss: 8.9603 - accuracy: 0.5242
Epoch 6/500
34/34 [==============================] - 0s 348us/step - loss: 8.6404 - accuracy: 0.5366
Epoch 7/500
34/34 [==============================] - 0s 337us/step - loss: 8.2321 - accuracy: 0.5408
Epoch 8/500
34/34 [==============================] - 0s 353us/step - loss: 8.0297 - accuracy: 0.5473
Epoch 9/500
34/34 [==============================] - 0s 361us/step - loss: 7.8792 - accuracy: 0.5479
Epoch 10/500
34/34 [==============================] - 0s 362us/step - loss: 7.7602 - acc

In [21]:
y_hat = model.predict(X_test)
print(y_hat[1])
y_test.head()

14/14 [==============================] - 0s 480us/step
[-0.13799687  3.8724995   8.844371   18.03471     1.8540289   7.9530706
  4.9650993   5.622057    8.73481     8.235362    1.3662189   0.86683065
  0.16942412  0.05660195]


,home_goals,away_goals,home_shots,away_shots,home_shots_on_target,away_shots_on_target,home_corners,away_corners,home_fouls,away_fouls,home_yellow_cards,away_yellow_cards,home_red_cards,away_red_cards
1634,2,1,13,16,4,3,3,5,8,10,1,1,0,0
1479,2,4,16,15,6,9,3,3,13,19,6,3,0,0
25,2,1,6,23,2,6,4,8,14,11,4,2,0,0
1686,2,1,12,17,4,5,6,8,11,7,3,1,0,0
1454,1,3,9,15,3,7,3,8,8,15,0,2,0,0


### n=20

In [22]:
n=20
X = n_components[n][['pca'+ str(x) for x in range(n)]]
Y = combined[output_columns]

X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size=0.2, random_state=629) 

In [23]:
model = tf.keras.models.Sequential([
	tf.keras.layers.Dense(32, activation='relu', input_dim=len(X_train.columns)),
	tf.keras.layers.Dense(128, activation='relu'),
	tf.keras.layers.Dense(OUTPUT_SIZE)
])
model.compile(loss="mean_squared_error", optimizer="sgd", metrics="accuracy")
model.fit(X_train, y_train, epochs=500, batch_size=50);

Epoch 1/500


34/34 [==============================] - 0s 564us/step - loss: 27.1094 - accuracy: 0.4255
Epoch 2/500
34/34 [==============================] - 0s 407us/step - loss: 12.6530 - accuracy: 0.4959
Epoch 3/500
34/34 [==============================] - 0s 396us/step - loss: 10.2278 - accuracy: 0.5118
Epoch 4/500
34/34 [==============================] - 0s 355us/step - loss: 9.1462 - accuracy: 0.5426
Epoch 5/500
34/34 [==============================] - 0s 649us/step - loss: 8.6268 - accuracy: 0.5502
Epoch 6/500
34/34 [==============================] - 0s 408us/step - loss: 8.2565 - accuracy: 0.5473
Epoch 7/500
34/34 [==============================] - 0s 434us/step - loss: 8.0915 - accuracy: 0.5561
Epoch 8/500
34/34 [==============================] - 0s 365us/step - loss: 7.8801 - accuracy: 0.5508
Epoch 9/500
34/34 [==============================] - 0s 363us/step - loss: 7.8102 - accuracy: 0.5491
Epoch 10/500
34/34 [==============================] - 0s 346us/step - loss: 7.6839 - accuracy: 0.554

In [24]:
y_hat = model.predict(X_test)
print(y_hat[1])
y_test.head()

14/14 [==============================] - 0s 340us/step
[ 0.14684656  2.4405859   2.8811357  18.124979    0.5322497   5.537651
  2.043296    7.0843854  14.035157   11.43758     2.191287    1.5889466
  0.13281788 -0.08902869]


,home_goals,away_goals,home_shots,away_shots,home_shots_on_target,away_shots_on_target,home_corners,away_corners,home_fouls,away_fouls,home_yellow_cards,away_yellow_cards,home_red_cards,away_red_cards
1634,2,1,13,16,4,3,3,5,8,10,1,1,0,0
1479,2,4,16,15,6,9,3,3,13,19,6,3,0,0
25,2,1,6,23,2,6,4,8,14,11,4,2,0,0
1686,2,1,12,17,4,5,6,8,11,7,3,1,0,0
1454,1,3,9,15,3,7,3,8,8,15,0,2,0,0


The above experiments show us the accuracy of each model increases with an increased number of features up to a point, it seem somewhere between 15 and 20 features, where the accuracy on the training data peaks. I will use 15 principal components going forward for the investigations, as this seems to yield the highest accuracy of those tested here

In terms of predicting exact values for the match facts, I am not expecting exact matches between model and the actual results, just hoping for a close approximation, or at the very least, the correct side with the higher value, i.e. home team scores more goals than the away team is correct

## 15 PC model build

#### Splitting data

In [25]:
X = combined[stats_columns]
y = combined[output_columns]

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=543)
print(X_train)

         goals   assists  penalties_scored  penalties_attempted  yellow_cards  \
618   0.282813  0.073807          0.012862             0.047048     -0.051462   
1581  1.264687  0.842364          0.022639             0.009442      0.984779   
1600  0.043288 -0.022940         -0.025762            -0.021131      0.497787   
812  -0.163778 -0.007934         -0.018938            -0.049900      0.391365   
2113 -0.150000 -0.039141         -0.012998            -0.012998     -0.092022   
...        ...       ...               ...                  ...           ...   
1542 -0.093617 -0.254741         -0.014639            -0.013673     -0.627743   
2062 -0.081567  0.127542         -0.037198            -0.056972      0.147570   
479  -0.660662 -0.268441         -0.115936            -0.129202     -0.484446   
561  -0.200871  0.054574          0.020231             0.000382     -0.386519   
1785 -0.501214 -0.169302         -0.016552            -0.023804     -0.299180   

      red_cards  expected_g

#### Scaling X and y

In [26]:
whole_x_scaler = StandardScaler(copy=True).fit(X)
whole_x_train = whole_x_scaler.transform(X)
dump(whole_x_scaler, '../prediction_scaler.bin')


# y_scaler = StandardScaler(copy=True).fit(y_train)

X_scaler = StandardScaler(copy=True).fit(X_train)
X_train = X_scaler.transform(X_train)
X_test = X_scaler.transform(X_test)


# y_train = y_scaler.transform(y_train)
# y_test = y_scaler.transform(y_test)

#### Applying PCA

In [27]:
# n=15
# print(X_train)
# pca = PCA(n_components = n, random_state=576)
# pca.fit(X)
# feature_to_pc_map = pd.DataFrame(pca.components_, columns=stats_columns)
# feature_to_pc_map.to_csv("../files/feature_to_15_pcs.csv")

# dump(pca, '../prediction_pca.bin')
# whole_X_pca = pca.transform(whole_x_train)

# pca = PCA(n_components = n, random_state=576)
# pca.fit(X_train)
# X_train = pca.transform(X_train)
# X_test = pca.transform(X_test)


# feature_to_pc_map

The below table shows the breakdown of each principle component in terms of the proportion of the value from each of the table columns used to make it up

In [28]:
df = pd.DataFrame(pca.components_, columns=stats_columns)
df.index.name = "principle_component_number"
df

,goals,assists,penalties_scored,penalties_attempted,yellow_cards,red_cards,expected_goals,non_penalty_expected_goals,expected_assisted_goals,progressive_carries,...,clean_sheets,penalties_faced,penalties_allowed,penalties_saved,penalties_missed,home_team_at_home_mean_goal_difference,home_team_overall_mean_goal_difference,away_team_at_away_mean_goal_difference,away_team_overall_mean_goal_difference,head_to_head_goal_difference
principle_component_number,,,,,,,,,,,,,,,,,,,,,
0,0.116647,0.130715,0.068400,0.070307,0.138751,0.070607,0.122651,0.123665,0.137644,0.138389,...,0.054194,0.060242,0.058161,0.050025,0.036246,0.007170,0.020208,-0.009295,-0.021773,0.010665
1,-0.097983,-0.088463,-0.059343,-0.063084,0.000732,0.013790,-0.092154,-0.091098,-0.084802,-0.080344,...,0.277699,0.287081,0.275237,0.238033,0.187666,0.010947,0.008860,-0.015492,-0.009881,0.006341
2,0.225147,0.119215,0.196468,0.200979,-0.100601,-0.071510,0.202254,0.189921,0.102687,0.072440,...,0.135467,0.152466,0.147709,0.124265,0.092075,-0.002510,0.014001,-0.004451,-0.011319,-0.006222
3,-0.032237,-0.007162,-0.056005,-0.054137,-0.025876,-0.010065,-0.029630,-0.023772,-0.005120,0.020702,...,0.000256,-0.000996,-0.016689,0.042219,0.040153,0.499535,0.326548,-0.456182,-0.302236,0.559459
4,-0.161452,0.084780,-0.547353,-0.538158,-0.063218,-0.062327,-0.157796,-0.083664,0.083214,0.160467,...,0.041171,0.042266,0.039578,0.020933,0.059715,-0.064370,0.009538,0.115516,0.016783,-0.074609
5,-0.007548,0.014139,-0.019727,-0.019406,-0.012637,0.002772,-0.011101,-0.008993,0.010566,0.004506,...,-0.000862,-0.004720,-0.010414,0.004729,0.025439,-0.362465,-0.570134,-0.409011,-0.609365,-0.017331
6,-0.137607,0.129524,0.137831,0.146285,-0.080254,-0.372094,-0.171512,-0.214617,0.128788,-0.002318,...,0.025236,-0.001838,-0.008773,0.033831,-0.011826,-0.009325,0.014917,0.022925,0.029258,-0.030050
7,0.090652,0.028161,-0.077109,-0.070137,-0.016359,-0.602959,0.098275,0.120059,0.014430,-0.008113,...,-0.026711,-0.046541,-0.032334,-0.026898,-0.142649,-0.099725,0.403521,0.101264,-0.388242,-0.191761
8,-0.098522,-0.094600,0.197279,0.194230,0.080062,0.144494,-0.079107,-0.119866,-0.049760,0.003162,...,0.010391,0.066956,0.052873,0.141550,-0.024785,-0.208141,0.366161,0.206319,-0.383752,-0.156610


#### Model Definition

In [29]:
INPUT_SIZE = len(X_train[0])
OUTPUT_SIZE = len(output_columns)

Define

In [30]:
model = tf.keras.models.Sequential([
	tf.keras.layers.Dense(INPUT_SIZE, activation='relu'),
	tf.keras.layers.Dense(12, activation='relu'),
	tf.keras.layers.Dense(OUTPUT_SIZE, activation='relu')
])

Compile

In [31]:
model.compile(loss="mean_squared_error", optimizer="sgd", metrics="accuracy")

Fit

In [32]:
model.fit(X_train, y_train, epochs=500, batch_size=25);

Epoch 1/500


68/68 [==============================] - 0s 980us/step - loss: 28.8963 - accuracy: 0.4196
Epoch 2/500
68/68 [==============================] - 0s 361us/step - loss: 19.2689 - accuracy: 0.4758
Epoch 3/500
68/68 [==============================] - 0s 320us/step - loss: 18.0975 - accuracy: 0.5165
Epoch 4/500
68/68 [==============================] - 0s 329us/step - loss: 17.4899 - accuracy: 0.5485
Epoch 5/500
68/68 [==============================] - 0s 357us/step - loss: 17.1969 - accuracy: 0.5437
Epoch 6/500
68/68 [==============================] - 0s 305us/step - loss: 16.8782 - accuracy: 0.5496
Epoch 7/500
68/68 [==============================] - 0s 325us/step - loss: 16.8098 - accuracy: 0.5485
Epoch 8/500
68/68 [==============================] - 0s 330us/step - loss: 16.7771 - accuracy: 0.5431
Epoch 9/500
68/68 [==============================] - 0s 319us/step - loss: 16.6620 - accuracy: 0.5544
Epoch 10/500
68/68 [==============================] - 0s 318us/step - loss: 16.6032 - accuracy

In [33]:
model.evaluate(X_test, y_test)

14/14 [==============================] - 0s 463us/step - loss: 10.7378 - accuracy: 0.4481


[10.737800598144531, 0.44811320304870605]

#### Testing

In [34]:
y_hat = model.predict(X_test)

14/14 [==============================] - 0s 438us/step


In [35]:
total_shots = 0
total_shots_over = 0
total_shots_under = 0

total_shots_on_target = 0
total_shots_on_target_over = 0
total_shots_on_target_under = 0

total_booking_points = 0
total_booking_points_over = 0
total_booking_points_under = 0

correct_score = 0

winner = 0

total_corners = 0
total_corners_over = 0
total_corners_under = 0

total_fouls = 0
total_fouls_over = 0
total_fouls_under = 0

goals_over = 0
goals_under = 0

all_under = 0
all_over = 0

total_tested  = len(y_hat)

In [36]:
for idx, y in enumerate(y_hat):
	home_goals_hat, away_goals_hat, home_shots_hat, away_shots_hat, home_shots_on_target_hat, away_shots_on_target_hat, home_corners_hat, away_corners_hat, home_fouls_hat, away_fouls_hat, home_yellow_cards_hat, away_yellow_cards_hat, home_red_cards_hat, away_red_cards_hat = y
	home_goals, away_goals, home_shots, away_shots, home_shots_on_target, away_shots_on_target, home_corners, away_corners, home_fouls, away_fouls, home_yellow_cards, away_yellow_cards, home_red_cards, away_red_cards = y_test.iloc[idx].tolist()
	
	total_shots += 1 if home_shots_hat+away_shots_hat == home_shots+away_shots else 0
	total_shots_over += 1 if home_shots_hat+away_shots_hat > home_shots+away_shots else 0
	total_shots_under += 1 if home_shots_hat+away_shots_hat < home_shots+away_shots else 0
	
	total_shots_on_target += 1 if home_shots_on_target_hat+away_shots_on_target_hat == home_shots_on_target+away_shots_on_target else 0
	total_shots_on_target_over += 1 if home_shots_on_target_hat+away_shots_on_target_hat > home_shots_on_target+away_shots_on_target else 0
	total_shots_on_target_under += 1 if home_shots_on_target_hat+away_shots_on_target_hat < home_shots_on_target+away_shots_on_target else 0

	total_booking_points += 1 if (home_yellow_cards_hat+away_yellow_cards_hat)*10+(home_red_cards_hat+away_red_cards_hat)*25 == (home_yellow_cards+away_yellow_cards)*10+(home_red_cards+away_red_cards)*25 else 0
	total_booking_points_over += 1 if (home_yellow_cards_hat+away_yellow_cards_hat)*10+(home_red_cards_hat+away_red_cards_hat)*25 > (home_yellow_cards+away_yellow_cards)*10+(home_red_cards+away_red_cards)*25 else 0
	total_booking_points_under += 1 if (home_yellow_cards_hat+away_yellow_cards_hat)*10+(home_red_cards_hat+away_red_cards_hat)*25 < (home_yellow_cards+away_yellow_cards)*10+(home_red_cards+away_red_cards)*25 else 0

	correct_score += 1 if home_goals_hat == home_goals and away_goals_hat == away_goals else 0
	
	winner += 1 if (home_goals_hat > away_goals_hat and home_goals > away_goals) or (home_goals_hat < away_goals_hat and home_goals < away_goals) else 0

	total_fouls += 1 if home_fouls_hat + away_fouls_hat == home_fouls + away_fouls else 0
	total_fouls_over += 1 if home_fouls_hat + away_fouls_hat > home_fouls+ away_fouls else 0
	total_fouls_under += 1 if home_fouls_hat + away_fouls_hat < home_fouls+ away_fouls else 0

	total_corners += 1 if home_corners_hat + away_corners_hat == home_corners + away_corners else 0
	total_corners_over += 1 if home_corners_hat + away_corners_hat > home_corners + away_corners else 0
	total_corners_under += 1 if home_corners_hat + away_corners_hat < home_corners + away_corners else 0

	goals_over += 1 if home_goals_hat + away_goals_hat > home_goals + away_goals else 0
	goals_under += 1 if home_goals_hat + away_goals_hat < home_goals + away_goals else 0

	all_under += 1 if home_shots_hat+away_shots_hat < home_shots+away_shots and home_shots_on_target_hat+away_shots_on_target_hat < home_shots_on_target+away_shots_on_target and (home_yellow_cards_hat+away_yellow_cards_hat)*10+(home_red_cards_hat+away_red_cards_hat)*25 < (home_yellow_cards+away_yellow_cards)*10+(home_red_cards+away_red_cards)*25 and home_corners_hat + away_corners_hat < home_corners + away_corners and home_corners_hat + away_corners_hat < home_corners + away_corners and home_goals_hat + away_goals_hat < home_goals + away_goals else 0
	all_over += 1 if home_shots_hat+away_shots_hat > home_shots+away_shots and home_shots_on_target_hat+away_shots_on_target_hat > home_shots_on_target+away_shots_on_target and (home_yellow_cards_hat+away_yellow_cards_hat)*10+(home_red_cards_hat+away_red_cards_hat)*25 > (home_yellow_cards+away_yellow_cards)*10+(home_red_cards+away_red_cards)*25 and home_corners_hat + away_corners_hat > home_corners + away_corners and home_goals_hat + away_goals_hat > home_goals + away_goals else 0

In [37]:
print("total_shots: "+str(total_shots/total_tested))
print("total_shots_over: "+str(total_shots_over/total_tested))
print("total_shots_under: "+str(total_shots_under/total_tested))
print("\n")
print("total_shots_on_target: "+str(total_shots_on_target/total_tested))
print("total_shots_on_target_over: "+str(total_shots_on_target_over/total_tested))
print("total_shots_on_target_under: "+str(total_shots_on_target_under/total_tested))
print("\n")
print("total_booking_points: "+str(total_booking_points/total_tested))
print("total_booking_points_over: "+str(total_booking_points_over/total_tested))
print("total_booking_points_under: "+str(total_booking_points_under/total_tested))
print("\n")
print("correct_score: "+str(correct_score/total_tested))
print("\n")
print("winner: "+str(winner/total_tested))
print("\n")
print("total_corners: "+str(total_corners/total_tested))
print("total_corners_over: "+str(total_corners_over/total_tested))
print("total_corners_under: "+str(total_corners_under/total_tested))
print("\n")
print("total_fouls: "+str(total_fouls/total_tested))
print("total_fouls_over: "+str(total_fouls_over/total_tested))
print("total_fouls_under: "+str(total_fouls_under/total_tested))
print("\n")
print("goals_over: "+str(goals_over/total_tested))
print("goals_under: "+str(goals_under/total_tested))
print("\n")
print("all_over: "+str(all_over/total_tested))
print("all_under: "+str(all_under/total_tested))

total_shots: 0.0
total_shots_over: 0.5047169811320755
total_shots_under: 0.49528301886792453


total_shots_on_target: 0.0
total_shots_on_target_over: 0.5306603773584906
total_shots_on_target_under: 0.4693396226415094


total_booking_points: 0.0
total_booking_points_over: 0.5141509433962265
total_booking_points_under: 0.4858490566037736


correct_score: 0.0


winner: 0.5966981132075472


total_corners: 0.0
total_corners_over: 0.4740566037735849
total_corners_under: 0.5259433962264151


total_fouls: 0.0
total_fouls_over: 0.589622641509434
total_fouls_under: 0.41037735849056606


goals_over: 0.5754716981132075
goals_under: 0.42452830188679247


all_over: 0.07783018867924528
all_under: 0.05188679245283019


### Tuning

Create a function to build and compile a model, with the arguments in it being the values to be update during tuning

In [38]:
def get_mlp_model(hidden_layer_one=13, dropout=0.2, learn_rate=0.01, n_h_layers=1):

	model = tf.keras.models.Sequential()

	# input
	model.add(tf.keras.layers.Dense(69, activation="relu", input_dim=69))

	for i in range(n_h_layers):
		model.add(tf.keras.layers.Dense(hidden_layer_one, activation="relu"))

	# dropout layer to remove redundant nodes
	model.add(tf.keras.layers.Dropout(dropout))
	
	# output
	model.add(tf.keras.layers.Dense(14, activation="relu"))

	model.compile(
		optimizer="sgd",
		loss="mse",
		metrics=["accuracy"])
	

	return model

Build a model using the *get_mlp_model* function, letting tensorflow use this function when building the models

In [39]:
model = KerasRegressor(model=get_mlp_model, verbose=0, hidden_layer_one=10, learn_rate=0.01, dropout=0.05, n_h_layers=1);

Define the hyperparameter search space, and then generate the search object to be used later, by RandomizedSearchCV

In [40]:
# define a grid of the hyperparameter search space
hidden_layer_one = [30]
learn_rate = [1e-2]
dropout = [0.1]
batch_size = [64]
epochs = [500]
n_h_layers = [3]

# create a dictionary from the hyperparameter grid
grid = dict(
	hidden_layer_one=hidden_layer_one,
	learn_rate=learn_rate,
	dropout=dropout,
	batch_size=batch_size,
	epochs=epochs,
	n_h_layers=n_h_layers
)

Build a scoring function for the searcher, average the value of all the *under* stats and return this

In [41]:
def scoring(estimator, test_x: np.ndarray, test_y: pd.DataFrame) -> float:
	test_y = test_y.to_numpy()
	
	# estimator.save("../stats_regression_model.h5")
	y_hat = estimator.predict(test_x)
	
	average_under_rate = 0

	total_shots_under = 0
	total_shots_on_target_under = 0
	total_booking_points_under = 0
	total_corners_under = 0
	total_fouls_under = 0
	goals_under = 0

	total_tested = len(y_hat)
	
	for idx, y in enumerate(y_hat):

		home_goals_hat, away_goals_hat, home_shots_hat, away_shots_hat, home_shots_on_target_hat, away_shots_on_target_hat, home_corners_hat, away_corners_hat, home_fouls_hat, away_fouls_hat, home_yellow_cards_hat, away_yellow_cards_hat, home_red_cards_hat, away_red_cards_hat = y
		home_goals, away_goals, home_shots, away_shots, home_shots_on_target, away_shots_on_target, home_corners, away_corners, home_fouls, away_fouls, home_yellow_cards, away_yellow_cards, home_red_cards, away_red_cards = test_y[idx]

		total_shots_under += 1 if np.floor(home_shots_hat)+np.floor(away_shots_hat) < home_shots+away_shots else 0
		total_shots_on_target_under += 1 if np.floor(home_shots_on_target_hat)+np.floor(away_shots_on_target_hat) < home_shots_on_target+away_shots_on_target else 0
		total_booking_points_under += 1 if (np.floor(home_yellow_cards_hat)+np.floor(away_yellow_cards_hat))*10+(np.floor(home_red_cards_hat)+np.floor(away_red_cards_hat))*25 < (home_yellow_cards+away_yellow_cards)*10+(home_red_cards+away_red_cards)*25 else 0
		total_corners_under += 1 if np.floor(home_corners_hat) + np.floor(away_corners_hat) < home_corners + away_corners else 0
		total_fouls_under += 1 if np.floor(home_fouls_hat) + np.floor(away_fouls_hat) < home_fouls+ away_fouls else 0
		goals_under += 1 if np.floor(home_goals_hat) + np.floor(away_goals_hat) < home_goals + away_goals else 0

		average_under_rate += ((total_shots_under/total_tested)+(total_shots_on_target_under/total_tested)+(total_booking_points_under/total_tested)+(total_fouls_under/total_tested)+(total_corners_under/total_tested)+(goals_under/total_tested))/6

	average_under_rate = average_under_rate/total_tested
	
	return average_under_rate

Generate the searcher object, used to iterate over the search parameters and determine the best hyper-parameter setup from the space we've created above

In [42]:
searcher = GridSearchCV(estimator=model, n_jobs=-2, param_grid=grid, scoring=scoring, verbose=4, cv=3, refit=True)

searchResults = searcher.fit(X_train, y_train)

bestScore = searchResults.best_score_
bestParams = searchResults.best_params_
print(f"[INFO] best score is {bestScore} using {bestParams}")

Fitting 3 folds for each of 1 candidates, totalling 3 fits


2024-04-29 09:49:53.337029: W tensorflow/tsl/platform/profile_utils/cpu_utils.cc:128] Failed to get CPU frequency: 0 Hz
2024-04-29 09:49:53.337054: W tensorflow/tsl/platform/profile_utils/cpu_utils.cc:128] Failed to get CPU frequency: 0 Hz
2024-04-29 09:49:53.337253: W tensorflow/tsl/platform/profile_utils/cpu_utils.cc:128] Failed to get CPU frequency: 0 Hz


[CV 1/3] END batch_size=64, dropout=0.1, epochs=500, hidden_layer_one=30, learn_rate=0.01, n_h_layers=3;, score=0.312 total time=   4.1s
[CV 2/3] END batch_size=64, dropout=0.1, epochs=500, hidden_layer_one=30, learn_rate=0.01, n_h_layers=3;, score=0.291 total time=   4.1s
[CV 3/3] END batch_size=64, dropout=0.1, epochs=500, hidden_layer_one=30, learn_rate=0.01, n_h_layers=3;, score=0.275 total time=   4.1s
[INFO] best score is 0.2924227277299936 using {'batch_size': 64, 'dropout': 0.1, 'epochs': 500, 'hidden_layer_one': 30, 'learn_rate': 0.01, 'n_h_layers': 3}


In [43]:
tuned_model = searcher.best_params_
tuned_model

{'batch_size': 64,
 'dropout': 0.1,
 'epochs': 500,
 'hidden_layer_one': 30,
 'learn_rate': 0.01,
 'n_h_layers': 3}

In [44]:
hidden_layer_one = tuned_model["hidden_layer_one"]
learn_rate = tuned_model["learn_rate"]
dropout = tuned_model["dropout"]
batch_size = tuned_model["batch_size"]
epochs = tuned_model["epochs"]
n_h_layers = tuned_model["n_h_layers"]

model = get_mlp_model(hidden_layer_one=30, learn_rate=0.01, dropout=0.1, n_h_layers=3)
model.fit(X_train, y_train, epochs=500, batch_size=64, )

model.save("../stats_regression_model.h5")
# pd.DataFrame(data=components[:, [p for p in range(15)]], columns=pca.get_feature_names_out(), )

Epoch 1/500


27/27 [==============================] - 0s 734us/step - loss: 30.7844 - accuracy: 0.3487
Epoch 2/500
27/27 [==============================] - 0s 498us/step - loss: 12.2725 - accuracy: 0.4492
Epoch 3/500
27/27 [==============================] - 0s 443us/step - loss: 10.6587 - accuracy: 0.4675
Epoch 4/500
27/27 [==============================] - 0s 417us/step - loss: 10.3095 - accuracy: 0.4793
Epoch 5/500
27/27 [==============================] - 0s 428us/step - loss: 9.5777 - accuracy: 0.4817
Epoch 6/500
27/27 [==============================] - 0s 427us/step - loss: 9.1755 - accuracy: 0.5100
Epoch 7/500
27/27 [==============================] - 0s 415us/step - loss: 9.0348 - accuracy: 0.5189
Epoch 8/500
27/27 [==============================] - 0s 417us/step - loss: 8.7631 - accuracy: 0.5449
Epoch 9/500
27/27 [==============================] - 0s 363us/step - loss: 8.5389 - accuracy: 0.5331
Epoch 10/500
27/27 [==============================] - 0s 368us/step - loss: 8.4135 - accuracy: 0.54

In [45]:
print(len
	  (X_train))

1692


## Next v  steps

- Tune parameters and layers on the 15 PC model to increase the accuracy
- Create a method to define the accuracy on the test set; strict equality for each of the output values is not necessary as under/over options on the betting platform mean a close approximation of the outcome is all that is required. Success/accuracy measure based on the bet markets available for each game:
	- Total shots
	- Total booking points
	- Correct score
	- Outright
	- Total corners
	- Under/over goals
- Save the model
- Complete the notebook with all annotations and text

RandomSearchCV  
best score is 0.39 using {'n_h_layers': 5, 'learn_rate': 0.0001, 'hidden_layer_one': 15, 'epochs': 100, 'dropout': 0.2, 'batch_size': 64}  
best score is 0.40 using {'n_h_layers': 7, 'learn_rate': 0.0001, 'hidden_layer_one': 15, 'epochs': 1000, 'dropout': 0.75, 'batch_size': 16}  
best score is 0.49 using {'n_h_layers': 3, 'learn_rate': 0.0001, 'hidden_layer_one': 30, 'epochs': 10, 'dropout': 0.3, 'batch_size': 64} 

GridSearchCV  
best score is 0.47 using {'batch_size': 64, 'dropout': 0.4, 'epochs': 100, 'hidden_layer_one': 10, 'learn_rate': 0.0001, 'n_h_layers': 2}  
best score is 0.49100 using {'batch_size': 64, 'dropout': 0.3, 'epochs': 10, 'hidden_layer_one': 10, 'learn_rate': 0.0001, 'n_h_layers': 3}  
best score is 0.49130 using {'batch_size': 64, 'dropout': 0.3, 'epochs': 20, 'hidden_layer_one': 15, 'learn_rate': 1e-05, 'n_h_layers': 4}  
best score is 0.49132 using {'batch_size': 64, 'dropout': 0.3, 'epochs': 20, 'hidden_layer_one': 15, 'learn_rate': 1e-06, 'n_h_layers': 4}  
best score is 0.49138 using {'batch_size': 64, 'dropout': 0.3, 'epochs': 15, 'hidden_layer_one': 10, 'learn_rate': 0.0001, 'n_h_layers': 6}  
best score is 0.49138 using {'batch_size': 64, 'dropout': 0.3, 'epochs': 15, 'hidden_layer_one': 10, 'learn_rate': 1e-05, 'n_h_layers': 6}  
best score is 0.49138 using {'batch_size': 64, 'dropout': 0.3, 'epochs': 15, 'hidden_layer_one': 10, 'learn_rate': 1e-07, 'n_h_layers': 6}  
best score is 0.49138 using {'batch_size': 64, 'dropout': 0.3, 'epochs': 15, 'hidden_layer_one': 10, 'learn_rate': 0.0001, 'n_h_layers': 4}  
best score is 0.49138 using {'batch_size': 32, 'dropout': 0.2, 'epochs': 12, 'hidden_layer_one': 10, 'learn_rate': 1e-05, 'n_h_layers': 4}  

- During
	- [CV 1/3] END batch_size=64, dropout=0.2, epochs=100, hidden_layer_one=15, learn_rate=0.0001, n_h_layers=1;, score=0.459 total time=   4.3s
	- [CV 3/3] END batch_size=64, dropout=0.2, epochs=100, hidden_layer_one=15, learn_rate=0.0001, n_h_layers=1;, score=0.462 total time=   3.2s
	- [CV 1/3] END batch_size=64, dropout=0.2, epochs=100, hidden_layer_one=15, learn_rate=0.0001, n_h_layers=2;, score=0.430 total time=   3.5s
	- [CV 2/3] END batch_size=64, dropout=0.2, epochs=100, hidden_layer_one=15, learn_rate=0.0001, n_h_layers=1;, score=0.458 total time=   5.3s
	- [CV 2/3] END batch_size=64, dropout=0.2, epochs=100, hidden_layer_one=15, learn_rate=0.0001, n_h_layers=2;, score=0.412 total time=   3.8s
	- [CV 3/3] END batch_size=64, dropout=0.2, epochs=100, hidden_layer_one=15, learn_rate=0.0001, n_h_layers=2;, score=0.451 total time=   3.5s
	- [CV 1/3] END batch_size=64, dropout=0.2, epochs=100, hidden_layer_one=15, learn_rate=0.0001, n_h_layers=3;, score=0.447 total time=   3.8s
	- [CV 2/3] END batch_size=64, dropout=0.2, epochs=100, hidden_layer_one=15, learn_rate=0.0001, n_h_layers=3;, score=0.413 total time=   4.8s
	- [CV 3/3] END batch_size=64, dropout=0.2, epochs=100, hidden_layer_one=15, learn_rate=0.0001, n_h_layers=3;, score=0.415 total time=   4.9s

## Questions

- Scaling y reduces the accuracy of the model by a factor of 6, why?